In [1]:
import requests
import smtplib
import pickle
import time
import glob
import ssl
import os

import pandas as pd
import numpy  as np

from collections    import defaultdict
from statistics     import mean
from random         import randint

# Foreword

This notebook was intially meant to be run using Tor, but some are understandably concerned with accessing the Tor network and using its services. I have updated this notebook from the class presentation to accommodate that and I still included the original notebook at the end.

# Introduction

We have a few goals with this notebook:
1. Scrape a website using BeautifulSoup
2. Use Tor to avoid IP detection
3. Randomly switch user agents to avoid detection
4. Schedule scraping to happen automatically every day
5. Email ourselves with information about scraping

The website that we'll be scraping can be found [here](http://www.vdc-sy.info/index.php/en/martyrs). They provide their information with in a dataframe-like structure, but unfortunately it doesn't display all the information they have per entry. If we click on a name we are brought to another page displaying more detailed information on the entry. We are lucky that the data we want to scrape from this site can be accessed directly by modifying the end of the url with a number up to around 250,000. 

# Libraries

Before anything, we'll need to install Tor and configure it.

In [ ]:
# !pip install tor

After installing Tor, we'll want to change our password associated with the Port that Tor uses. 

Make sure to copy the hashed password!

In [ ]:
# !tor --hash-password cmps184

Now we need to make some changes to our Tor configuration file. In Mac OS, it can be found in usr/etc/tor/torrc

We have to uncomment lines 57, 60, and 61.

In the HashedControlPassword field, paste the hashed password you just got earlier.

cd usr/local/etc/tor/torrc

You can view hidden files in MacOS by holding cmd shift and pressing + . 

Torrequests is the library that we'll use in order to send requests over the Tor Network. It is a very simple wrapper for the regular requests library.

In [ ]:
# !pip install torrequest

Finally we can install fake-useragent, which will let us cycle through new useragents.

In [ ]:
# !pip install fake_useragent

We can now install the schedule library, which will need to be used for traditional or Tor-based scraping.

In [ ]:
!pip install schedule

# Traditional

These cells will allow you to run the scraping code without having to use Tor. There is no commentary because this notebook was meant to use Tor originally.

In [ ]:
def save(obj, name):
    pickle.dump(obj, open(name + '.pickle', 'wb'))

def load(name):
    return pickle.load(open(name + '.pickle', 'rb'))

In [ ]:
import re
from   bs4            import BeautifulSoup as bs

def scrape_recent():
    first_page = 'http://www.vdc-sy.info/index.php/en/martyrs/1/c29ydGJ5PWEua2lsbGVkX2RhdGV8c29ydGRpcj1ERVNDfGFwcHJvdmVkPXZpc2libGV8ZXh0cmFkaXNwbGF5PTB8'
    pattern    = re.compile('\/index\.php\/en\/details\/martyrs\/.')

    try:
        response = requests.get(first_page)
        content  = bs(response.text, 'html.parser')
        links    = {link['href'][30:] for link in content.find_all('a', href = True) if pattern.match(link['href'])} 

    except Exception as e:
        print(e)

    return links

In [ ]:
def scrape_details(uid):
    cols = []
    vals = []

    url  = 'http://www.vdc-sy.info/index.php/en/details/martyrs/' + uid
    page = requests.get(url).text
    page = bs(page, 'html.parser')
    
    table = page.find('table', attrs = {'class':'peopleListing'})
    rows  = table.find_all('tr')

    for row in rows:
        data = row.find_all('td')

        # All data without only 2 data values
        # are not data we are looking for
        if len(data) != 2:
            continue

        # data[0] corresponds to the row label/column
        cols.append(data[0].text)
        
        # Values need to appended differently for image rows 
        if data[1].find('img') is not None:
            vals.append(data[1].find('img')['src'])
        else:
            vals.append(data[1].text)

    # Adds the uid to the dataframe
    cols.append('uid')
    vals.append(uid)

    # Creates and saves dataframe
    person = pd.DataFrame([vals], columns = cols, dtype=str)
    save(person, uid)

In [ ]:
"""
Adapted from https://realpython.com/python-send-email/
"""

import ssl
import smtplib

def get_password():
    # This is the actual password for this account, so please do not share this notebook outside of staff
    # I will change the password after grades are submitted.
    #
    # The reason I am doing this is to allow you to send this email
    # without you ahving to setup a throwaway account for yourself
    
    return 'dipping1_mitzvoth'

def send_email():
    # Either of these ports may work. Port 465 works for my computer.
    # but Port 443 works for others
    
#     port     = 465
    port     = 443
    
    password = get_password()

    sender_email   = "romanlosul@gmail.com"  
    receiver_email = "rsul@ucsc.edu"  

    message = """\
    Subject: VDC Scrape Log

    Scraping was a success.
    """

    context = ssl.create_default_context()
    with smtplib.SMTP_SSL("smtp.gmail.com", port, context=context) as server:
        server.login(sender_email, password)
        server.sendmail(sender_email, receiver_email, message)

In [ ]:
def job():
    
    # We will load in the uids that we've already scraped
    # and scrape the first page to get recent ids
    scraped_uids   = load('scraped_uids')
    recent_uids    = scrape_recent()
    uids_to_scrape = []
    
    # We'll add any ids that we haven't already seen
    # And then scrape the detailed pages
    for uid in recent_uids:
        if uid not in scraped_uids:
            uids_to_scrape.append(uid)
    
    # We pop a uid off our list of uids to scrape and
    # attempt to scrape it the detailed page. If it fails, 
    # then it adds it back to the queue and resets the UserAgent
    # and Tor identity.
    #
    # If it succeeds then it adds the uid to the list of scraped uids 
    # and saves that file to load in later
    
    while len(uids_to_scrape) > 0:
        uid = uids_to_scrape.pop()

        try:
            scrape_details(uid)

        except Exception as e:
            # Most of the time this script will fail
            # due to a requests issue. So we will sleep
            # for 5 seconds.
            
            print(e)
            uids_to_scrape.append(uid)
            time.sleep(5)
        
            continue

        print('Left to scrape: ', len(uids_to_scrape))
        scraped_uids.add(uid)
        save(scraped_uids, 'scraped_uids')
        
    send_email()

In [ ]:
import schedule

schedule.every(1).to(23).hours.do(job)

In [ ]:
while True:
    schedule.run_pending()

# Tor Based Scraping

# Scraping

First we'll write a couple helper functions that make it easier to save files locally. This will let us resume progress if any errors kill our script. These files will be saved to the local directory as pickle files.

In [20]:
def save(obj, name):
    pickle.dump(obj, open(name + '.pickle', 'wb'))

def load(name):
    return pickle.load(open(name + '.pickle', 'rb'))

Now for the scraping. This will be relatively easy. We will get the page using a TorRequests to mask our IP, navigate to the data we need using BeautifulSoup, create a dataframe from that data, and save it to a file.

The function scrape_recent() will handle the scraping for the most recent ids. It does this by finding all links that match a regex pattern and then checking these against a list of all previous ids. We'll use the next function to handle the rest.

The function scrape_details() will handle scraping the pages on the website that contain the detailed information of an entry. This represents **one** person. Each person is saved and returned as a dataframe, since a person will have varying amounts and types of labels/columns and will have to be combined into one large dataset later. 

In [21]:
import re

from fake_useragent import UserAgent
from torrequest     import TorRequest
from bs4            import BeautifulSoup as bs

def scrape_recent():
    first_page = 'http://www.vdc-sy.info/index.php/en/martyrs/1/c29ydGJ5PWEua2lsbGVkX2RhdGV8c29ydGRpcj1ERVNDfGFwcHJvdmVkPXZpc2libGV8ZXh0cmFkaXNwbGF5PTB8'
    pattern    = re.compile('\/index\.php\/en\/details\/martyrs\/.')

    ua         = UserAgent()
    headers    = {'User-Agent': ua.random}
    tor        = TorRequest(password = 'cmps184')
    tor.reset_identity()

    try:
        response = tor.get(first_page, headers=headers)
        content  = bs(response.text, 'html.parser')
        
        links    = {link['href'][30:] for link in content.find_all('a', href = True) if pattern.match(link['href'])} 

    except Exception as e:
        print(e)

    return links


In [22]:
scrape_recent()

{'204066',
 '204067',
 '204068',
 '204069',
 '204070',
 '204071',
 '204072',
 '204073',
 '204074',
 '204075',
 '204076',
 '204078',
 '204081',
 '204084',
 '204085',
 '204087',
 '204088',
 '204089',
 '204090',
 '204091',
 '204092',
 '204093',
 '204094',
 '204095',
 '204096',
 '204097',
 '204098',
 '204099',
 '204100',
 '204102',
 '204103',
 '204104',
 '204105',
 '204106',
 '204107',
 '204108',
 '204110',
 '204111',
 '204112',
 '204113',
 '204114',
 '204116',
 '204117',
 '204141',
 '204142',
 '204149',
 '204150',
 '204151',
 '204152',
 '204153',
 '204154',
 '204155',
 '204156',
 '204157',
 '204158',
 '204159',
 '204160',
 '204161',
 '204162',
 '204163',
 '204164',
 '204165',
 '204166',
 '204167',
 '204168',
 '204169',
 '204170',
 '204171',
 '204172',
 '204173',
 '204174',
 '204175',
 '204176',
 '204177',
 '204178',
 '204179',
 '204181',
 '204182',
 '204183',
 '204184',
 '204185',
 '204186',
 '204187',
 '204189',
 '204190',
 '204191',
 '204192',
 '204193',
 '204194',
 '204195',
 '204196',

In [23]:
def scrape_details(uid, tor, headers):
    cols = []
    vals = []

    url  = 'http://www.vdc-sy.info/index.php/en/details/martyrs/' + uid
    
    # Headers will provide the UserAgent to use when getting response
    # Makes the request using a TorRequest object passed in
    page = tor.get(url, headers = headers).text
    page = bs(page, 'html.parser')
    
    table = page.find('table', attrs = {'class':'peopleListing'})
    rows  = table.find_all('tr')

    for row in rows:
        data = row.find_all('td')

        # All data without only 2 data values
        # are not data we are looking for
        if len(data) != 2:
            continue

        # data[0] corresponds to the row label/column
        cols.append(data[0].text)
        
        # Values need to appended differently for image rows 
        if data[1].find('img') is not None:
            vals.append(data[1].find('img')['src'])
        else:
            vals.append(data[1].text)

    # Adds the uid to the dataframe
    cols.append('uid')
    vals.append(uid)

    # Creates and saves dataframe
    person = pd.DataFrame([vals], columns = cols, dtype=str)
#     print(person.head())
    save(person, uid)

Now we can prepare the function to send an email. We'll have to change our Google Account settings first. We'll then have to turn on the 'Less secure app access' in the Security tab. This will allow us to log in and send emails through an account using python.

Both the sll and smtplib librariers are part of standard Python.

In [27]:
"""
Adapted from https://realpython.com/python-send-email/
"""
import ssl
import smtplib

def get_password():
    return 'dipping1_mitzvoth'

def send_email():
    # Either of these ports may work. Port 465 works for my computer.
    # but Port 443 works for others
    
#     port     = 465
#     port     = 443
    
    password = get_password()

    sender_email   = "romanlosul@gmail.com"  
    
    # I will actually receive these emails. I don't mind it sin
    receiver_email = "rsul@ucsc.edu"  

    message = """\
    Subject: VDC Scrape Log

    Scraping was a success.
    """

    context = ssl.create_default_context()
    with smtplib.SMTP_SSL("smtp.gmail.com", port, context=context) as server:
        server.login(sender_email, password)
        server.sendmail(sender_email, receiver_email, message)

Now that this function seems ready to go, we can execute it to test whether it works

In [28]:
send_email()

Using Tor without changing our user agent is useless, since we can still be identified by our User Agent.

We will use the [fake-useragent](https://github.com/hellysmile/fake-useragent) library to cycle through random user agents. There are issues with other approaches using User Agents that are out of date or uncommon, but this library User Agent selection is based on usage statistics from [http://useragentstring.com/](http://useragentstring.com/).

Our process flow can be described as:
    1. Get a random UserAgent
    2. Create TorRequest instance
    3. Remove uid from Queue
    4. Scrape details with uid
    5. Save progress
    5. Repeat 3-5 until Queue is empty

In [29]:
def job():
    
    # This is slightly modified from the UserAgent docs
    # It creates a UserAgent Object, and assigns
    # a random UserAgent to a header dict
    
    ua         = UserAgent()
    headers    = {'User-Agent': ua.random}
    
    # Setting up Torrequest
    # Followed instructions from 
    # https://www.scrapehero.com/make-anonymous-requests-using-tor-python/
    
    # This password is what we used earlier when setting up Tor
    # TorRequest gives us a new IP address
    tor = TorRequest(password = 'commonhorse')
    
    # reset_identity() should reset our IP Address, but 
    # it currently has a bug. We will keep it here for when 
    # TorRequests is updated. In the mean time, instantiate 
    # a new TorRequest object to get a new IP address.
    tor.reset_identity()
    
    # We will load in the uids that we've already scraped
    # and scrape the first page to get recent ids
    scraped_uids   = load('scraped_uids')
    recent_uids    = scrape_recent()
    uids_to_scrape = []
    
    # We'll add any ids that we haven't already seen
    # And then scrape the detailed pages
    for uid in recent_uids:
        if uid not in scraped_uids:
            uids_to_scrape.append(uid)
    
    # We pop a uid off our list of uids to scrape and
    # attempt to scrape it the detailed page. If it fails, 
    # then it adds it back to the queue and resets the UserAgent
    # and Tor identity.
    #
    # If it succeeds then it adds the uid to the list of scraped uids 
    # and saves that file to load in later
    
    while len(uids_to_scrape) > 0:
        uid = uids_to_scrape.pop()

        try:
            scrape_details(uid, tor, headers)

        except Exception as e:
            # Most of the time this script will fail
            # due to a requests issue. So we reset our
            # useragent and Tor instance.
            
            print(e)
            uids_to_scrape.add(uid)
            
            ua         = UserAgent()
            headers    = {'User-Agent': ua.random}
            tor = TorRequest(password = 'commonhorse')
            tor.reset_identity()
        
            continue

        print('Left to scrape: ', len(uids_to_scrape))
        scraped_uids.add(uid)
        save(scraped_uids, 'scraped_uids')
        
    send_email()

The scheduling library is meant to be used easily. You can actually set it at these intervals: 

    second
    minute
    hour
    day
    week
    monday
    tuesday
    wednesday
    thursday
    friday
    saturday
    sunday

We can now schedule our scraping to happen at midnight everyday, so long as this notebook is up and running.

In [19]:
import schedule

schedule.every().day.at("00:00").do(job)

Every 1 day at 00:00:00 do job() (last run: [never], next run: 2019-06-05 00:00:00)

This may be problematic for us though, since we are scraping at the same time every day we are more noticeable to the site. So instead we'll scrape at a random time everyday. One way to do this is to generate a random time to scrape. 

In [ ]:
# from random import randint

# hour   = f'{randint(0 , 23):02}'
# minute = f'{randint(0 , 59):02}'
# rand_t = f'{hour}:{minute}'

# schedule.every().day.at(rand_t).do(job)

We can do this using the 'to' command instead. 

In [ ]:
schedule.every(1).to(23).hours.do(job)

And now with this command this code will continue to run so long as this notebook is open.

In [ ]:
while True:
    schedule.run_pending()

In [30]:
job()

Left to scrape:  46
Left to scrape:  45
Left to scrape:  44
Left to scrape:  43
Left to scrape:  42
Left to scrape:  41
Left to scrape:  40
Left to scrape:  39
Left to scrape:  38
Left to scrape:  37
Left to scrape:  36
Left to scrape:  35


KeyboardInterrupt: 